In [79]:
import pandas as pd #Dataframe, Series
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import precision_recall_curve

%matplotlib inline

# Spotify Login

In [80]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

cid ="c8c47267a0d14b7cbb529001dd126978" 
secret = "41d0284be0d24f5aa9e5b7a6ff3fc039"
username = "12156070492"
redirect_uri = "http://localhost:8888/"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-modify-private playlist-read-private'

token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

# Input Data to the Model
- You need to include the user you want to get the playlist from as well as the playlist id's of your good and bad playlists.
- To do this go to your good and bad playlists and copy the links.
    - An example spotify link: https://open.spotify.com/user/1287242681/playlist/5OdH7PmotfAO7qDGxKdw3J
    - The user is the number after user/ and the playlist id is after the playlist/.

- The method signature is sp.user_playlist('user', 'playlist_id')

In [81]:
# Tutorials
# good_playlist = sp.user_playlist("1287242681", "5OdH7PmotfAO7qDGxKdw3J")
# bad_playlist = sp.user_playlist("1287242681", "3ySDAXYGUwRrp8C4ejIm9m")

# Megan's
# good_playlist = sp.user_playlist("oa66aggjdka7torff2helltk6", "1rArXwbIyQvU31gzc5XHHY?si=6oQTTUWTQ4-xizaDju06ug")
# bad_playlist = sp.user_playlist("oa66aggjdka7torff2helltk6", "13bLJeS4U2Drj8SCcwAH24?si=1-aMqHL4ST69DJL1Ho6ZwA")

# My Playlists
good_playlist = sp.user_playlist("12156070492", "1CEtpLF6xMXR4ODsMBNjhv")
bad_playlist = sp.user_playlist("12156070492", "0dgHltQuMc4vsRw76U9C1N")

In [ ]:
#Get the good song id's out of the good playlists, need it for the audio features call.
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
print(len(good_songs))
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])

In [77]:
#Now do the same thing for the bad playlist
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
print(len(bad_songs))
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])

1665


In [78]:
#Here is where we get all of the audio_features for the tracks on the good and bad playlists.
features = []
inSavedTracks = []
j = 0
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 1
j = 0
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 0

AttributeError: 'NoneType' object has no attribute 'split'

# Model Training and Data Modeling
- Now that we have the data on what songs you like and don't like let's visualize some of that info.

In [ ]:
# trainingData = pd.DataFrame(features)
# trainingData.head()
features

Models using my own dataset

In [ ]:
df = pd.DataFrame(features)
filename = 'arcy_music.csv'
df.to_csv(filename, index=False, encoding='utf-8')
trainingData = pd.read_csv("arcy_music.csv")
trainingData.head()

- In training this classifier, we need so split our data into training and testing data so our classifier has things to train on and then test if it is right

In [ ]:
train, test = train_test_split(trainingData, test_size = 0.20)
print("Training size: {}, Test size: {}".format(len(train),len(test)))

In [ ]:
# Custom Color Palette for graphs
red_blue = ['#19B5FE', '#EF4836']
palette = sns.color_palette(red_blue)
sns.set_palette(palette)
sns.set_style('white')

# Graphs!!!!!!!

1) First, break out all of the data into positive and negative categories for all the features we want to compare

Also, I know there has to be a better way of doing this, but copy paste is my friend. :)

In [ ]:
pos_tempo = trainingData[trainingData['target'] == 1]['tempo']
neg_tempo = trainingData[trainingData['target'] == 0]['tempo']
pos_dance = trainingData[trainingData['target'] == 1]['danceability']
neg_dance = trainingData[trainingData['target'] == 0]['danceability']
pos_duration = trainingData[trainingData['target'] == 1]['duration_ms']
neg_duration = trainingData[trainingData['target'] == 0]['duration_ms']
pos_loudness = trainingData[trainingData['target'] == 1]['loudness']
neg_loudness = trainingData[trainingData['target'] == 0]['loudness']
pos_speechiness = trainingData[trainingData['target'] == 1]['speechiness']
neg_speechiness = trainingData[trainingData['target'] == 0]['speechiness']
pos_valence = trainingData[trainingData['target'] == 1]['valence']
neg_valence = trainingData[trainingData['target'] == 0]['valence']
pos_energy = trainingData[trainingData['target'] == 1]['energy']
neg_energy = trainingData[trainingData['target'] == 0]['energy']
pos_acousticness = trainingData[trainingData['target'] == 1]['acousticness']
neg_acousticness = trainingData[trainingData['target'] == 0]['acousticness']
pos_key = trainingData[trainingData['target'] == 1]['key']
neg_key = trainingData[trainingData['target'] == 0]['key']
pos_instrumentalness = trainingData[trainingData['target'] == 1]['instrumentalness']
neg_instrumentalness = trainingData[trainingData['target'] == 0]['instrumentalness']
pos_popularity = trainingData[trainingData['target'] == 1]['trackPopularity']
neg_popularity = trainingData[trainingData['target'] == 0]['trackPopularity']

In [ ]:
print(pos_popularity)
print(neg_popularity)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.title("Song Tempo Like / Dislike Distribution")
pos_tempo.hist(alpha=0.7, bins=30, label='positive')
neg_tempo.hist(alpha=0.7, bins=30, label='negative')
plt.legend(loc='upper right')

In [ ]:
fig2 = plt.figure(figsize=(15,15))

#Danceability
ax3 = fig2.add_subplot(331)
ax3.set_xlabel('Danceability')
ax3.set_ylabel('Songs')
ax3.set_title('Danceability Distribution')
pos_dance.hist(alpha= 0.7, bins=50, color='0.75')
ax4 = fig2.add_subplot(331)
neg_dance.hist(alpha= 0.3, bins=50, color='r')

#Duration_ms
ax5 = fig2.add_subplot(332)
ax5.set_xlabel('Duration')
ax5.set_ylabel('Songs')
ax5.set_title('Duration Distribution')
pos_duration.hist(alpha= 0.7, bins=50, color='0.75')
ax6 = fig2.add_subplot(332)
neg_duration.hist(alpha= 0.3, bins=50, color='r')

#Loudness
ax7 = fig2.add_subplot(333)
ax7.set_xlabel('Loudness')
ax7.set_ylabel('Songs')
ax7.set_title('Loudness Distribution')
pos_loudness.hist(alpha= 0.7, bins=50, color='0.75')
ax8 = fig2.add_subplot(333)
neg_loudness.hist(alpha= 0.3, bins=50, color='r')

#Key
ax9 = fig2.add_subplot(334)
ax9.set_xlabel('Key')
ax9.set_ylabel('Songs')
ax9.set_title('Key Distribution')
pos_key.hist(alpha= 0.7, bins=50, color='0.75')
ax10 = fig2.add_subplot(334)
neg_key.hist(alpha= 0.3, bins=50, color='r')

#Valence
ax11 = fig2.add_subplot(335)
ax11.set_xlabel('Valence')
ax11.set_ylabel('Songs')
ax11.set_title('Valence Distribution')
pos_valence.hist(alpha= 0.7, bins=50, color='0.75')
ax12 = fig2.add_subplot(335)
neg_valence.hist(alpha= 0.3, bins=50, color='r')

#Energy
ax13 = fig2.add_subplot(336)
ax13.set_xlabel('Energy')
ax13.set_ylabel('Songs')
ax13.set_title('Energy Distribution')
pos_energy.hist(alpha= 0.7, bins=50, color='0.75')
ax14 = fig2.add_subplot(336)
neg_energy.hist(alpha= 0.3, bins=50, color='r')

#Speechiness
ax15 = fig2.add_subplot(337)
ax15.set_xlabel('Speechiness')
ax15.set_ylabel('Songs')
ax15.set_title('Speechiness Distribution')
pos_speechiness.hist(alpha= 0.7, bins=50, color='0.75')
ax16 = fig2.add_subplot(337)
neg_speechiness.hist(alpha= 0.3, bins=50, color='r')

#Popularity
ax15 = fig2.add_subplot(338)
ax15.set_xlabel('Popularity')
ax15.set_ylabel('Songs')
ax15.set_title('Popularity Distribution')
pos_popularity.hist(alpha= 0.7, bins=50, color='0.75')
ax16 = fig2.add_subplot(338)
neg_popularity.hist(alpha= 0.3, bins=50, color='r')

In [ ]:
plt.rc("legend", fontsize=35)
#sns.jointplot(y=df['ENERGY'], x=df['LOUD'], kind="hex", color="r", size=13).set_axis_labels("Loudness", "Energy", fontsize=35)
#sns.jointplot(y=df['ENERGY'], x=df['ACOUSTIC'], kind="hex", color="r", size=13).set_axis_labels("Acousticness", "Energy", fontsize=35)
#sns.jointplot(y=df['DANCE'], x=df['VALENCE'], kind="hex", color="r", size=13).set_axis_labels("Valence (positive mood)", "Danceability", fontsize=35)
#sns.jointplot(y=df['ACOUSTIC'], x=df['LOUD'], kind="hex", color="r", size=13).set_axis_labels("Loudness", "Acousticness", fontsize=35)
#sns.jointplot(y=df['ENERGY'], x=df['VALENCE'], kind="hex", color="r", size=13).set_axis_labels("Valence (positive mood)", "Energy", fontsize=35)
#sns.jointplot(y=df['POP.'], x=df['ENERGY'], kind="hex", color="r", size=13).set_axis_labels("Energy", "Popularity", fontsize=35)
sns.jointplot(y=trainingData['acousticness'], x=trainingData['danceability'], kind="hex", color="b", size=13).set_axis_labels("Dance", "Acoustic", fontsize=35)

# Wewh, now that we have graphs galore, lets actually get into some different classifiers and see how they preform!

In [ ]:
#Define the set of features that we want to look at
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness","duration_ms"]

In [ ]:
#Split the data into x and y test and train sets to feed them into a bunch of classifiers!
x_train = train[features]
y_train = train["target"]

x_test = test[features]
y_test = test["target"]

1) A Decision Tree Classifier
- A decision tree classifier is often the easiest to visualize, so we will start with it first
- All it is is pretty much a decision tree based off the features so you can trace the path down and visually see how it makes decisions. This is nice to visualize but it isn't all that good at predicting this data.

In [ ]:
c = DecisionTreeClassifier(min_samples_split=100)
dt = c.fit(x_train, y_train)

In [ ]:
def show_tree(InputTree, features, path):
    f = io.StringIO()
    tree.export_graphviz(InputTree, out_file=f, feature_names=features)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
    img = misc.imread(path)
    plt.rcParams["figure.figsize"] = (20,20)
    plt.imshow(img)
show_tree(dt, features, "dec_tree.png")

This is how we will see how our different classifiers do, by predicting the results on the test data and then figuring out how well they did.

In [ ]:
y_pred = c.predict(x_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 1), "%")

In [ ]:
# DecisionTree precision
precision_score(y_test, y_pred)

In [ ]:
# DecisionTree recall
recall_score(y_test, y_pred)

In [ ]:
target_names = ['good', 'bad']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
confusion_matrix(y_test, y_pred, labels=[1,0])

# A Bunch More Classifier Models
- There are some of these that take a bunch of time to finish, I wouldnt recommend running all them unless you are really intrested in the results

## The ones that take less time

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(3)
knn_fit = knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)
acc = accuracy_score(y_test, knn_pred) * 100
prec = precision_score(y_test, knn_pred) * 100
recall = recall_score(y_test, knn_pred) * 100
conf = confusion_matrix(y_test, knn_pred, labels=[1,0])
print("Accuracy using Knn Tree: ", round(acc, 1), "%")
print("Precision score Knn Tree:", round(prec, 1))
print("Recall score Knn Tree:", round(recall, 1), "%")
print("Confusion Matrix score:\n", conf)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(x_train, y_train)
mlp_pred = mlp.predict(x_test)
score = accuracy_score(y_test, mlp_pred) * 100
prec = precision_score(y_test, mlp_pred) * 100
recall = recall_score(y_test, mlp_pred) * 100
print("Accuracy using MLP Tree: ", round(score, 1), "%")
print("Precision Score using MLP Tree: ", round(prec, 1), "%")
print("Recall Score using MLP Tree: ", round(recall, 1), "%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
forest.fit(x_train, y_train)
forest_pred = forest.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, forest_pred) * 100
prec = precision_score(y_test, forest_pred) * 100
recall = recall_score(y_test, forest_pred) * 100
print("Accuracy using random forest: ", round(score, 1), "%")
print("Precision Score using random forest: ", round(prec, 1), "%")
print("Recall Score using random forest: ", round(recall, 1), "%")
target_names = ['good', 'bad']
print(classification_report(y_test, forest_pred, target_names=target_names))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, ada_pred) * 100
prec = precision_score(y_test, ada_pred) * 100
recall = recall_score(y_test, ada_pred) * 100
print("Accuracy using ada: ", round(score, 1), "%")
print("Preciscion Score using ada: ", round(prec, 1), "%")
print("Recall Score using ada: ", round(recall, 1), "%")
target_names = ['good', 'bad']
print(classification_report(y_test, ada_pred, target_names=target_names))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gauss = GaussianNB()
gauss.fit(x_train, y_train)
gauss_pred = gauss.predict(x_test)
score = accuracy_score(y_test, gauss_pred)*100
prec = precision_score(y_test, gauss_pred)*100
recall = recall_score(y_test, gauss_pred)*100
print("Accuracy using gauss: ", round(score, 1), "%")
print("Accuracy using gauss: ", round(prec, 1), "%")
print("Accuracy using gauss: ", round(recall, 1), "%")
target_names = ['good', 'bad']
print(classification_report(y_test, gauss_pred, target_names=target_names))

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=3, random_state=0)
k_means.fit(x_train, y_train)
predicted= k_means.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Kmeans: ", round(score, 1), "%")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)
predicted = gbc.predict(x_test)
score = accuracy_score(y_test, predicted)*100
prec = precision_score(y_test, predicted)*100
recall = recall_score(y_test, predicted)*100
print("Accuracy using Gbc: ", round(score, 1), "%")
print("Precision using Gbc: ", round(prec, 1), "%")
print("Recall using Gbc: ", round(recall, 1), "%")

In [ ]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier()
bc.fit(x_train, y_train)
predicted = bc.predict(x_test)
score = accuracy_score(y_test, predicted)*100
prec = precision_score(y_test, predicted)*100
recall = recall_score(y_test, predicted)*100
print("Accuracy using bc: ", round(score, 1), "%")
print("Precision using bc: ", round(prec, 1), "%")
print("Recall using bc: ", round(recall, 1), "%")

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

clf1 = GradientBoostingClassifier(learning_rate=0.2, max_depth=10, n_estimators=300)
clf2 = BaggingClassifier(n_estimators=100)
# lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2]
                          , meta_classifier=clf1
                          , use_probas=True
                          , average_probas=False,)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, sclf], 
                      ['GBC', 
                       'BC',
                       'StackingClassifier']):

    scores = cross_val_score(clf, x_train, y_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

## The ones that take more time

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)
qda_pred = qda.predict(x_test)
score = accuracy_score(y_test, qda_pred)*100
print("Accuracy using qda: ", round(score, 1), "%")

In [ ]:
from sklearn.svm import SVC
svc_lin = SVC(kernel="linear", C=0.025)
svc_lin.fit(x_train, y_train)
svc_pred = svc_lin.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, svc_pred) * 100
print("Accuracy using svc linear: ", round(score, 1), "%")

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
gpc = GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True)
gpc.fit(x_train, y_train)
gpc_pred = gpc.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, gpc_pred) * 100
print("Accuracy using gpc: ", round(score, 1), "%")

# Now we should have a classifier with the highest accuracy score

## Now we need to pick a playlist you want you classifier to find songs you like in...
- Use the same thing you used to find the good and bad playlists to specify this new playlist you've taken an intrest in.

In [ ]:
playlistToFindSongsYouLikeIn = sp.user_playlist("spotify", "37i9dQZEVXcFqDuJHHirGk")

newPlaylist_tracks = playlistToFindSongsYouLikeIn["tracks"]
newPlaylist_songs = newPlaylist_tracks["items"] 
while newPlaylist_tracks['next']:
    newPlaylist_tracks = sp.next(newPlaylist_tracks)
    for song in newPlaylist_tracks["items"]:
        newPlaylist_songs.append(song)
        
newPlaylist_song_ids = [] 
print(len(newPlaylist_songs))
for i in range(len(newPlaylist_songs)):
    newPlaylist_song_ids.append(newPlaylist_songs[i]['track']['id'])
    
newPlaylist_features = []
j = 0
for i in range(0,len(newPlaylist_song_ids),50):
    audio_features = sp.audio_features(newPlaylist_song_ids[i:i+50])
    for track in audio_features:
        track['song_title'] = newPlaylist_songs[j]['track']['name']
        track['artist'] = newPlaylist_songs[j]['track']['artists'][0]['name']
        j= j + 1
        newPlaylist_features.append(track)
print(len(newPlaylist_features))

playlistToLookAtFeatures = pd.DataFrame(newPlaylist_features)

# We just loaded in your new tracks and audio features for those tracks
- Now all we have to do is find the highest accuracy value out of all you classifiers (for me it was gdc) and predict what we will have over the new playlist

### Quick aside: you can put these into a new playlist as well, just use the sp.user_playlist_add_tracks function
- The function looks like this user_playlist_add_tracks("username", "playlist_id", "track_id_to_add")

In [ ]:
pred = gbc.predict(playlistToLookAtFeatures[features])

likedSongs = 0
i = 0
for prediction in pred:
    if(prediction == 1):
        print ("Song: " + playlistToLookAtFeatures["song_title"][i] + ", By: "+ playlistToLookAtFeatures["artist"][i])
        #sp.user_playlist_add_tracks("1287242681", "7eIX1zvtpZR3M3rYFVA7DF", [test['id'][i]])
        likedSongs= likedSongs + 1
    i = i +1
    

# There Ya Go!
- This was my quick overview of my process of how I used classifiers and spoitfy to try and help me find tracks I might like in any playlist on spotify!
- I want to make this tutorial less involved and figure out a way to make my classifiers better as well as using Tensorflow to make a model in order to do this for me.

# Tuning using GridSearchCV

## I. GradientBoostingClassifier

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = Pipeline([("imputer", SimpleImputer(strategy='mean'))
                      , ('standardizer', StandardScaler())
                      , ("gbm", GradientBoostingClassifier())
                     ])

parameters = {'gbm__n_estimators': [50, 100, 300]
              , 'gbm__max_depth' : [8, 10, 12]
              , 'gbm__learning_rate' : [0.1, 0.2, 0.7]
             }

In [ ]:
gs = GridSearchCV(estimator, parameters, cv=3, verbose=3, scoring='roc_auc', n_jobs=1)
gs.fit(x_train, y_train.values.ravel())
gs.best_params_

tuned_estimator = Pipeline([("imputer", SimpleImputer(strategy='mean'))
                      , ('standardizer', StandardScaler())
                      , ("gbm", GradientBoostingClassifier(learning_rate=0.2
                                                          , max_depth=10
                                                          , n_estimators=300))
                     ])

tuned_estimator.fit(x_train, y_train.values.ravel())

predicted = tuned_estimator.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Tuned GBC: ", round(score, 1), "%")

## II. BaggingClassifier

In [ ]:
estimator = Pipeline([("imputer", SimpleImputer(strategy='mean'))
                      , ('standardizer', StandardScaler())
                      , ("bc", BaggingClassifier())
                     ])

parameters = {'bc__n_estimators': [50, 100, 300]}

gs = GridSearchCV(estimator, parameters, cv=3, verbose=3, scoring='roc_auc', n_jobs=1)
gs.fit(x_train, y_train.values.ravel())
gs.best_params_

In [ ]:
tuned_estimator = Pipeline([("imputer", SimpleImputer(strategy='mean'))
                      , ('standardizer', StandardScaler())
                      , ("bc", BaggingClassifier(n_estimators=100))
                     ])

tuned_estimator.fit(x_train, y_train.values.ravel())


predicted = tuned_estimator.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Tuned BC: ", round(score, 1), "%")